In [ ]:
# 개 고양이 사진데이터
# 데이터 구성
# 총 3022개
# train : 2000개
# validation : 1000개
# test : 22개

In [1]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [2]:
import os

In [3]:
# 각각 폴더 경로 적기
train_dir = './test/'
val_dir = './test/'
test_dir = './test/'

In [4]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(val_dir)))
print(len(os.listdir(test_dir)))

2
2
2


In [5]:
train_bi_dir = train_dir+"비빔밥/train"
train_bul_dir = train_dir+"불고기/train"

test_bi_dir = test_dir+"비빔밥/test"
test_bul_dir = test_dir+"불고기/test"

val_bi_dir = val_dir +"비빔밥/val"
val_bul_dir = val_dir +"불고기/val"

In [6]:
print(len(os.listdir(train_bi_dir)))
print(len(os.listdir(train_bul_dir)))
print(len(os.listdir(val_bi_dir)))
print(len(os.listdir(val_bul_dir)))
print(len(os.listdir(test_bi_dir)))
print(len(os.listdir(test_bul_dir)))

2000
2000
500
500
1
1


In [7]:
# 이미지 데이터를 하나의 변수에 집어넣기
# 픽셀값 255 > 1
# 이미지 크기 (150,150) 으로 변환
# 라벨링
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 픽셀값 0 ~ 1사이 변환
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

# flow_from_directory() : 디렉토리로부터 이미지를 가져오는 함수
# flow_from_directory(디렉토리 이름, 변환할 이미지 크기, 한번에 변환할 이미지 수, 라벨링)

# binary = 이진분류
# 폴더를 알파벳 순으로 라벨링, 0의 값부터 라벨링
# cats 폴더 안에 있는 이미지들은 0
# dogs 폴더 안에 있는 이미지들은 1

# categorical = 다중분류
train_generator = train_gen.flow_from_directory(train_dir,
                                                target_size = (150,150),
                                                batch_size = 100,
                                                class_mode = 'binary')

val_generator = val_gen.flow_from_directory(val_dir,
                                                target_size = (150,150),
                                                batch_size = 100,
                                                class_mode = 'binary')

test_generator = test_gen.flow_from_directory(test_dir,
                                                target_size = (150,150),
                                                batch_size = 100,
                                                class_mode = 'binary')


Found 5002 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


In [8]:
# 라벨링 결과 확인
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

{'불고기': 0, '비빔밥': 1}
{'불고기': 0, '비빔밥': 1}
{'불고기': 0, '비빔밥': 1}


In [9]:
# CNN 모델 설계

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

model1 = Sequential()

# 입력층
# 특징 추출부
model1.add(Conv2D(
    filters = 32,
    kernel_size = (3,3),
    input_shape = (150,150, 3),
    padding = 'same',
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size = (2,2)
))

model1.add(Flatten()) # MLP이 학습하기 위해서 데이터를 1차원으로 만들어주는 역할

############################# 특징 추출부 끝 ################
############################# 분류부 시작 ###################
# 이미 위에서 특징들을 추출했기 때문에 층을 깊게 쌓지 않아도 됨
# MLP 사용
model1.add(Dense(units = 512 , activation = 'relu'))
# 과대적합 방지
# 학습할때는 0.3만큼 drop해서 0.7만큼의 유닛만 사용해서 학습
# 예측할때는 전부 사용해서 예측
model1.add(Dropout(0.3))

# 출력층
model1.add(Dense(units = 1 , activation = 'sigmoid'))

model1.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 180000)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               92160512  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 92,161,921
Trainable params: 92,161,921
Non-trainable params: 0
____________________________________________

In [10]:
# 이진분류
model1.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [ ]:
model1.fit_generator(
    generator = train_generator,
    # steps_per_epoch = 10,
    epochs = 20,
    validation_data = val_generator,
    # validation_steps = 10
)

In [ ]:
# 이미지 증식
# imagedatagenerator에서 사용 가능
train_gen1 = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator1 = train_gen1.flow_from_directory(train_dir,
                                                target_size = (150,150),
                                                batch_size = 20,
                                                class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [ ]:
# 이진분류
model1.compile(loss = 'binary_crossentropy',
            optimizer = 'adam',
            metrics = ['accuracy'])

In [ ]:
model1.fit_generator(
    generator = train_generator1,
    # steps_per_epoch = 20,
    epochs = 20,
    validation_data = val_generator,
    # validation_steps = 20
)

In [19]:
# VGG16 모델을 사용해서 전이학습
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet', # imagenet에 있는 가중치 사용
                  include_top = False, # 분류기를 사용 할것인가 ? > 분류기는 우리가 설계
                  input_shape = (150,150,3)
    )

58900480/58889256 [==============================] - 1s 0us/step


In [20]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
model2 = Sequential()

# 설계할 모델에 VGG16끼워넣기

model2.add(conv_base)

model2.add(Flatten())

model2.add(Dense(units = 512 ,activation = 'relu'))
model2.add(Dense(units = 1 ,activation = 'sigmoid'))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 18,910,017
Trainable params: 18,910,017
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 동결
# 오차 역전파시에 VGG16모델의 가중치가 갱신되지 않도록 막는것

In [22]:
# 동결되기전에 학습되는 가중치수
print(len(model2.trainable_weights))

30


In [23]:
# VGG16의 전체 층에 대해 동결
conv_base.trainable = False

In [ ]:
# 동결 후의 훈련되는 가중치수
print(len(model2.trainable_weights))

4


In [24]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [25]:
model2.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

Epoch 1/20
51/51 [==============================] - 300s 6s/step - loss: 0.8716 - accuracy: 0.6881 - val_loss: 0.3704 - val_accuracy: 0.8439
Epoch 2/20
51/51 [==============================] - 301s 6s/step - loss: 0.3048 - accuracy: 0.8762 - val_loss: 0.2198 - val_accuracy: 0.9248
Epoch 3/20
37/51 [====================>.........] - ETA: 42s - loss: 0.2130 - accuracy: 0.9189

KeyboardInterrupt: 

In [ ]:
# 미세조정
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
model3 = Sequential()

# 설계할 모델에 VGG16끼워넣기

model3.add(conv_base)

model3.add(Flatten())

model3.add(Dense(units = 512 ,activation = 'relu'))
model3.add(Dense(units = 1 ,activation = 'sigmoid'))

model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 18,910,017
Trainable params: 18,910,017
Non-trainable params: 0
_________________________________________________________________


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
# block5_conv1 ~ 3 층을 학습 시켜보자
conv_base.trainable = False

# VGG 신경망 한층(block5_conv3)를 가져와서 학습하게 만들어보자

set_trainable = False

In [ ]:
for layer in conv_base.layers:
  # 가져온 층의 이름이 block5_conv1이라면
  # block5_conv1 하위의 층들을 학습을 하도록 동결을 풀어준다
  if layer.name == 'block4_conv2':
    set_trainable = True
  
  if set_trainable == True:
    layer.trainable = True
  else :
    layer.trainable = False

In [ ]:
model3.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [ ]:
model3.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
20/20 [==============================] - 18s 881ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.3663 - val_accuracy: 0.9050
Epoch 2/20
20/20 [==============================] - 17s 858ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.3556 - val_accuracy: 0.9100
Epoch 3/20
20/20 [==============================] - 17s 856ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.3697 - val_accuracy: 0.9060
Epoch 4/20
20/20 [==============================] - 17s 854ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.3680 - val_accuracy: 0.9060
Epoch 5/20
20/20 [==============================] - 17s 849ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.3695 - val_accuracy: 0.9090
Epoch 6/20
20/20 [==============================] - 17s 850ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.3763 - val_accuracy: 0.9090
Epoch 7/20
20/20 [==============================] - 17s 861ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.3894 - val_accuracy: 0.9060